In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CZ4041/Code-Practice/trial_database_balanced")
!ls

annotation.txt					 test.zip
frcnn_test_vgg.ipynb				 train
frcnn_train_resnet.ipynb			 train_annotation.csv
frcnn_train_vgg.ipynb				 train_annotation.xlsx
model						 train.csv
model_resnet_config.pickle			 train_ImageID.txt
model_vgg_config.pickle				 val_annotation.txt
Object_Detection_DataPreprocessing.ipynb	 val.csv
Preprocessingv2.ipynb				 validation_annotation.csv
subset_Object_Detection_DataPreprocessing.ipynb  validation_annotation.xlsx
test						 validation_ImageID.txt
test_annotation.xlsx				 val.zip


In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
tf.__version__

In [ ]:
# save images to train and test directory
train_path = 'train'
test_path = 'test'
val_path='val'

In [ ]:
classes = ['Man', 'Boy','Woman','Girl']

Training Annotation 

In [ ]:
train_annotations_bbox_fname = 'train_annotation.csv'
train_annotations_bbox = pd.read_csv(train_annotations_bbox_fname)
train_annotations_bbox.head()

,ImageID,Category,XMin,XMax,YMin,YMax
0,0001c3e14fe71e81,Man,0.262387,0.541667,0.227599,1.000000
1,0001c3e14fe71e81,Boy,0.028153,0.143018,0.252688,0.983871
2,0001c3e14fe71e81,Boy,0.145270,0.288288,0.333333,0.998208
3,0001c3e14fe71e81,Boy,0.183559,0.322072,0.458781,0.998208
4,0001c3e14fe71e81,Boy,0.235360,0.382883,0.370968,0.863799


In [ ]:
!mkdir train test

mkdir: cannot create directory ‘train’: File exists


In [ ]:
random.seed(1)

In [ ]:
label_names = ['Man', 'Boy', 'Woman', 'Girl']

train_df = pd.DataFrame(columns=['ImageID','Category', 'XMin', 'XMax', 'YMin', 'YMax', ])

# Find boxes in each image and put them in a dataframe
train_imgs = os.listdir(train_path)
train_imgs = [name for name in train_imgs if not name.startswith('.')]

for i in range(len(train_imgs)):
    sys.stdout.write('Parse train_imgs ' + str(i) + '; Number of boxes: ' + str(len(train_df)) + '\r')
    sys.stdout.flush()
    img_name = train_imgs[i]
    img_id = img_name[0:16]
    tmp_df = train_annotations_bbox[train_annotations_bbox['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['Category']
        for i in range(len(label_names)):
            if labelName == label_names[i]:
                train_df = train_df.append({'ImageID': img_name, 
                                            'XMin': row['XMin'], 
                                            'XMax': row['XMax'], 
                                            'YMin': row['YMin'], 
                                            'YMax': row['YMax'], 
                                            'Category': row['Category']}, 
                                           ignore_index=True)

In [ ]:
train_df.head()
train_df.shape
train_df.to_csv('train.csv')

Validation set 

In [ ]:
test_annotations_bbox_fname = 'test_annotation.csv'
test_annotations_bbox = pd.read_csv(test_annotations_bbox_fname)
test_annotations_bbox.head()

,ImageID,Category,XMin,XMax,YMin,YMax
0,00156ac2ee0c3d58,Man,0.257732,0.639175,0.057522,0.891593
1,0038b12aeccdcb7e,Man,0.154867,0.402655,0.433628,0.873894
2,0038b12aeccdcb7e,Man,0.564897,0.728614,0.203540,0.867257
3,005304a16107b370,Girl,0.092920,0.917404,0.000000,0.745575
4,005304a16107b370,Girl,0.092920,0.141593,0.002212,0.050885


In [ ]:
label_names = ['Man', 'Boy', 'Woman', 'Girl']

test_df = pd.DataFrame(columns=['ImageID','Category', 'XMin', 'XMax', 'YMin', 'YMax', ])

# Find boxes in each image and put them in a dataframe
test_imgs = os.listdir(test_path)
test_imgs = [name for name in test_imgs if not name.startswith('.')]

for i in range(len(test_imgs)):
    sys.stdout.write('Parse test_imgs ' + str(i) + '; Number of boxes: ' + str(len(test_df)) + '\r')
    sys.stdout.flush()
    img_name = test_imgs[i]
    img_id = img_name[0:16]
    tmp_df = test_annotations_bbox[test_annotations_bbox['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['Category']
        for i in range(len(label_names)):
            if labelName == label_names[i]:
                test_df = test_df.append({'ImageID': img_name, 
                                            'XMin': row['XMin'], 
                                            'XMax': row['XMax'], 
                                            'YMin': row['YMin'], 
                                            'YMax': row['YMax'], 
                                            'Category': row['Category']}, 
                                           ignore_index=True)

In [ ]:
print(test_df.head())
print(test_df.shape)
test_df.to_csv('test.csv')

                ImageID Category      XMin      XMax      YMin      YMax
0  5126c87695b9c344.jpg    Woman  0.000000  0.780236  0.190265  0.995575
1  5139400fff7bb583.jpg     Girl  0.000000  0.677621  0.191667  1.000000
2  5139400fff7bb583.jpg     Girl  0.000000  0.242567  0.308333  1.000000
3  5139400fff7bb583.jpg     Girl  0.447574  0.687011  0.243750  1.000000
4  5139400fff7bb583.jpg     Girl  0.654147  0.940532  0.275000  1.000000
(1563, 6)


Testing Set 

In [ ]:
val_annotations_bbox_fname = 'validation_annotation.csv'
val_annotations_bbox = pd.read_csv(val_annotations_bbox_fname)
val_annotations_bbox.head()

,ImageID,Category,XMin,XMax,YMin,YMax
0,000a1249af2bc5f0,Woman,0.000000,1.000000,0.000000,1.000000
1,000a1249af2bc5f0,Man,0.222059,0.998529,0.000000,0.988914
2,00141571d986d241,Girl,0.140625,0.751563,0.312500,0.879167
3,00141571d986d241,Woman,0.139063,0.751563,0.343750,0.881250
4,00141571d986d241,Man,0.000000,0.176563,0.164583,1.000000


In [ ]:
label_names = ['Man', 'Boy', 'Woman', 'Girl']

val_df = pd.DataFrame(columns=['ImageID','Category', 'XMin', 'XMax', 'YMin', 'YMax', ])

# Find boxes in each image and put them in a dataframe
val_imgs = os.listdir(val_path)
val_imgs = [name for name in val_imgs if not name.startswith('.')]

for i in range(len(val_imgs)):
    sys.stdout.write('Parse val_imgs ' + str(i) + '; Number of boxes: ' + str(len(val_df)) + '\r')
    sys.stdout.flush()
    img_name = val_imgs[i]
    img_id = img_name[0:16]
    tmp_df = val_annotations_bbox[val_annotations_bbox['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['Category']
        for i in range(len(label_names)):
            if labelName == label_names[i]:
                val_df = val_df.append({'ImageID': img_name, 
                                            'XMin': row['XMin'], 
                                            'XMax': row['XMax'], 
                                            'YMin': row['YMin'], 
                                            'YMax': row['YMax'], 
                                            'Category': row['Category']}, 
                                           ignore_index=True)

In [ ]:
print(val_df.head())
print(val_df.shape)
val_df.to_csv('val.csv')

                ImageID Category      XMin      XMax      YMin      YMax
0  000a1249af2bc5f0.jpg    Woman  0.000000  1.000000  0.000000  1.000000
1  000a1249af2bc5f0.jpg      Man  0.222059  0.998529  0.000000  0.988914
2  00173afdc7581c41.jpg      Man  0.000000  0.448661  0.116071  0.967262
3  00173afdc7581c41.jpg      Man  0.408482  0.997768  0.000000  0.997024
4  00146ba1e50ed8d8.jpg      Boy  0.000000  0.593548  0.000000  0.885417
(4864, 6)


**This looks fine to me!!**

### Write train.csv to annotation.txt

In [ ]:
train_df = pd.read_csv('train.csv')

# for training
with open("annotation.txt", "w+") as f:
  for idx, row in train_df.iterrows():
      img = cv2.imread('train/' + row['ImageID'])
      height, width = img.shape[:2]
      x1 = int(row['XMin'] * width)
      x2 = int(row['XMax'] * width)
      y1 = int(row['YMin'] * height)
      y2 = int(row['YMax'] * height)
      
      google_colab_file_path = '/content/drive/My Drive/Colab Notebooks/CZ4041/Code-Practice/trial_database_balanced/train'
      fileName = os.path.join(google_colab_file_path, row['ImageID'])
      className = row['Category']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

In [ ]:
test_df = pd.read_csv('test.csv')

# for test
with open("test_annotation.txt", "w+") as f:
  for idx, row in test_df.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      img = cv2.imread('test/' + row['ImageID'])
      height, width = img.shape[:2]
      x1 = int(row['XMin'] * width)
      x2 = int(row['XMax'] * width)
      y1 = int(row['YMin'] * height)
      y2 = int(row['YMax'] * height)
      
      google_colab_file_path = '/content/drive/My Drive/Colab Notebooks/CZ4041/Code-Practice/trial_database_balanced/test'
      fileName = os.path.join(google_colab_file_path, row['ImageID'])
      className = row['Category']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

In [ ]:
val_df = pd.read_csv('val.csv')

# for test
with open("val_annotation.txt", "w+") as f:
  for idx, row in val_df.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      img = cv2.imread('val/' + row['ImageID'])
      height, width = img.shape[:2]
      x1 = int(row['XMin'] * width)
      x2 = int(row['XMax'] * width)
      y1 = int(row['YMin'] * height)
      y2 = int(row['YMax'] * height)
      
      google_colab_file_path = '/content/drive/My Drive/Colab Notebooks/CZ4041/Code-Practice/trial_database_balanced/val'
      fileName = os.path.join(google_colab_file_path, row['ImageID'])
      className = row['Category']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')